In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
family_group = "8.Lycaenidae"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
    # print(coordinate_info_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Aricia_agestis
Exon_1 Match
[['Aricia_agestis', ['21824941', '21825037']], ['Danaus_plexippus', ['21824941', '21825037']], ['Maniola_jurtina', ['21824941', '21825037']]]
Exon_2 Match
[['Aricia_agestis', ['21826059', '21826609']], ['Danaus_plexippus', ['21826059', '21826609']], ['Maniola_jurtina', ['21826059', '21826609']]]
Exon_3 Match
[['Aricia_agestis', ['21827200', '21827353']], ['Danaus_plexippus', ['21827200', '21827353']], ['Maniola_jurtina', ['21827200', '21827353']]]
Exon_4 Match
[['Aricia_agestis', ['21830559', '21830779']], ['Danaus_plexippus', ['21830559', '21830779']], ['Maniola_jurtina', ['21830559', '21830779']]]
Exon_5 mismatch
[['Aricia_agestis', ['21831475', '21831722']], ['Danaus_plexippus', ['21831505', '21831722']], ['Maniola_jurtina', ['21831475', '21831722']]]
Proceed?y
Aricia_agestis Exon_5 96_frame0 0.0 
DLLAKDTVMKEKLYSDKKNKRNIEFISRRKNQETTTTSSNEDELPAKKPHNSKPRMTSKQRPPSATPQDRKRKKIVKRGKLN
Danaus_plexippus Exon_5 96_frame0 1.272 
EKLYSDKKNKRNIEFISRRKNQETTTTSSNEDELPA

Exon_1 Match
[['Aricia_agestis', ['23375642', '23375738']], ['Danaus_plexippus', ['23375642', '23375738']], ['Maniola_jurtina', ['23375642', '23375738']]]
Exon_2 Match
[['Aricia_agestis', ['23376884', '23377434']], ['Danaus_plexippus', ['23376884', '23377434']], ['Maniola_jurtina', ['23376884', '23377434']]]
Exon_3 Match
[['Aricia_agestis', ['23378020', '23378173']], ['Danaus_plexippus', ['23378020', '23378173']], ['Maniola_jurtina', ['23378020', '23378173']]]
Exon_4 Match
[['Aricia_agestis', ['23382763', '23382986']], ['Danaus_plexippus', ['23382763', '23382986']], ['Maniola_jurtina', ['23382763', '23382986']]]
Exon_5 mismatch
[['Aricia_agestis', ['23383689', '23383936']], ['Danaus_plexippus', ['23383719', '23383936']], ['Maniola_jurtina', ['23383689', '23383936']]]
Proceed?y
Aricia_agestis Exon_5 94_frame2 0.0 
DLLAKDTVMKEKLYSDKKNKRNIEFISRRKNQETTTTSSNEDELPAKKPHNSKPRMTSKQRPPSATPQDRKRKKIVKRGKLN
Danaus_plexippus Exon_5 94_frame2 1.277 
EKLYSDKKNKRNIEFISRRKNQETTTTSSNEDELPAKKPHNSKPRMTSKQR

Exon_1 Match
[['Aricia_agestis', ['289272', '289368']], ['Danaus_plexippus', ['289272', '289368']], ['Maniola_jurtina', ['289272', '289368']]]
Exon_2 Match
[['Aricia_agestis', ['287384', '287934']], ['Danaus_plexippus', ['287384', '287934']], ['Maniola_jurtina', ['287384', '287934']]]
Exon_3 Match
[['Aricia_agestis', ['286353', '286506']], ['Danaus_plexippus', ['286353', '286506']], ['Maniola_jurtina', ['286353', '286506']]]
Exon_4 mismatch
[['Aricia_agestis', ['284276', '284574']], ['Danaus_plexippus', ['284327', '284574']], ['Maniola_jurtina', ['284276', '284574']]]
Proceed?y
Aricia_agestis Exon_4 62_frame2 1.256 
SDLSSPILTSDPTSESSDAVNSGQSDEDLVSNKPCQDAPKNVEEIKTESIDEKQKNDALNNEIQHINTISDETEQKNSENKPVYPSITKFNNIKESCGC
Danaus_plexippus Exon_4 62_frame2 1.437 
SDLSSPILTSDPTSESSDAVNSGQSDEDLVSNKPCQDAPKNVEEIKTESIDEKQKNDALNNEIQHINTISDETEQKNSENKP
Maniola_jurtina Exon_4 62_frame2 1.519 
SDLSSPILTSDPTSESSDAVNSGQSDEDLVSNKPCQDAPKNVEEIKTESIDEKQKNDALNNEIQHINTISDETEQKNSENKPVYPSITKFNNIKESCGC
Check Alignm

Exon_1 Match
[['Aricia_agestis', ['8698243', '8698339']], ['Danaus_plexippus', ['8698243', '8698339']], ['Maniola_jurtina', ['8698243', '8698339']]]
Exon_2 Match
[['Aricia_agestis', ['8699182', '8699732']], ['Danaus_plexippus', ['8699182', '8699732']], ['Maniola_jurtina', ['8699182', '8699732']]]
Exon_3 Match
[['Aricia_agestis', ['8700324', '8700477']], ['Danaus_plexippus', ['8700324', '8700477']], ['Maniola_jurtina', ['8700324', '8700477']]]
Exon_4 mismatch
[['Aricia_agestis', ['8700847', '8701058']], ['Danaus_plexippus', ['8700847', '8701058']], ['Maniola_jurtina', ['8700847', '8701112']]]
Proceed?y
Aricia_agestis Exon_4 29_frame2 0.626 
SDHSSPILTSDSSNTVGSGKSDEELPNTTWADTTRENKQINNEMQQKAMDEKSAYTSSENDHNIKESSES
Danaus_plexippus Exon_4 29_frame2 1.555 
SDHSSPILTSDSSNTVGSGKSDEELPNTTWADTTRENKQINNEMQQKAMDEKSAYTSSENDHNIKESSES
Maniola_jurtina Exon_4 29_frame2 1.601 
SDHSSPILTSDSSNTVGSGKSDEELPNTTWADTTRENKQINNEMQQKAMDEKSAYTSSENDHNIKESSESVSFIRTSEKGFTISIIKI
Check Alignment??y
CLUSTAL format alignm

Exon_1 Match
[['Aricia_agestis', ['6452749', '6452845']], ['Danaus_plexippus', ['6452749', '6452845']], ['Maniola_jurtina', ['6452749', '6452845']]]
Exon_2 Match
[['Aricia_agestis', ['6450925', '6451475']], ['Danaus_plexippus', ['6450925', '6451475']], ['Maniola_jurtina', ['6450925', '6451475']]]
Exon_3 Match
[['Aricia_agestis', ['6450186', '6450339']], ['Danaus_plexippus', ['6450186', '6450339']], ['Maniola_jurtina', ['6450186', '6450339']]]
Exon_4 Match
[['Aricia_agestis', ['6449393', '6449616']], ['Danaus_plexippus', ['6449393', '6449616']], ['Maniola_jurtina', ['6449393', '6449616']]]
Exon_5 Match
[['Aricia_agestis', ['6448813', '6449060']], ['Danaus_plexippus', ['6448813', '6449060']], ['Maniola_jurtina', ['6448813', '6449060']]]
Exon_6 Match
[['Aricia_agestis', ['6448194', '6448554']], ['Danaus_plexippus', ['6448194', '6448554']], ['Maniola_jurtina', ['6448194', '6448554']]]
Exon_7 Match
[['Aricia_agestis', ['6444667', '6444858']], ['Danaus_plexippus', ['6444667', '6444858']], ['

...

Proceed?y
Aricia_agestis Exon_10 72_frame0 0.531 
DWSDIIEYAINTFVKKPHKLYLNSSNMCLEPETLEDKKHKDKDNTDDAEVATHDTKKSNLSNTKH
Danaus_plexippus Exon_10 72_frame0 1.268 
DWSDIIEYAINTFVKKPHKLYLNSSNMCLEPETLEDKKHKDKDNTDDAEVATHDTK
Maniola_jurtina Exon_10 72_frame0 1.433 
DWSDIIEYAINTFVKKPHKLYLNSSNMCLEPETLEDKKHKDKDNTDDAEVATHDTKKSNLSNTKH
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ DWSDMIEYAINTFVKKPHKLYLNSSNMRLEAESPEDKKH--KDNTDDAEIATLD-RKSVL
set72_frame0    DWSDIIEYAINTFVKKPHKLYLNSSNMCLEPETLEDKKHKDKDNTDDAEVATHDTKKSNL
                ****:********************** **.*: *****  ********:** * :** *

Aricia_agestis_ SYTK-D
set72_frame0    SNTKHK
                * ** .

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Danaus_plexippu DWSDLIEYVINTFIKKPYKLQ-NYSNIQLEMEIFDDDM-KPKENLIDPEPAPEIISVMKK
set72_frame0    DWSDIIEYAINTFVKKPHKLYLNSSNMCLEPETLEDKKHKDKDNTDDAEVATHD---TKK
                ****:***.****:***:**  * **: ** * ::*.  * *:*  *.* *..     **

Danaus_plexippu VNE

Choose Species 1 - 31
Exon_6 Match
[['Aricia_agestis', ['6978436', '6978796']], ['Danaus_plexippus', ['6978436', '6978796']], ['Maniola_jurtina', ['6978436', '6978796']]]
Exon_7 Match
[['Aricia_agestis', ['6977168', '6977359']], ['Danaus_plexippus', ['6977168', '6977359']], ['Maniola_jurtina', ['6977168', '6977359']]]
Exon_8 Match
[['Aricia_agestis', ['6976294', '6976391']], ['Danaus_plexippus', ['6976294', '6976391']], ['Maniola_jurtina', ['6976294', '6976391']]]
Exon_9 Match
[['Aricia_agestis', ['6974760', '6974958']], ['Danaus_plexippus', ['6974760', '6974958']], ['Maniola_jurtina', ['6974760', '6974958']]]
Exon_10 mismatch
[['Aricia_agestis', ['6960115', '6960302']], ['Danaus_plexippus', ['6960133', '6960302']], ['Maniola_jurtina', ['6960115', '6960302']]]
Proceed?y
Aricia_agestis Exon_10 334_frame0 0.074 
DWSDIIEYAINTFVKKPHKLYLNSSNMRLEAESSEDKKHKDNTDDAEIATLDRKSVLSYTKD
Danaus_plexippus Exon_10 334_frame0 1.192 
DWSDIIEYAINTFVKKPHKLYLNSSNMRLEAESSEDKKHKDNTDDAEIATLDRKSV
Maniola_jurtina

Choose Species 1 - 3n
Exon_12 Match
[['Aricia_agestis', ['6950840', '6951040']], ['Danaus_plexippus', ['6950840', '6951040']], ['Maniola_jurtina', ['6950840', '6951040']]]
Exon_13 Match
[['Aricia_agestis', ['6949462', '6949633']], ['Danaus_plexippus', ['6949462', '6949633']], ['Maniola_jurtina', ['6949462', '6949633']]]
Exon_14 Match
[['Aricia_agestis', ['6947553', '6947651']], ['Danaus_plexippus', ['6947553', '6947651']], ['Maniola_jurtina', ['6947553', '6947651']]]
Exon_15 mismatch
[['Aricia_agestis', ['6947151', '6947309']], ['Danaus_plexippus', ['6949462', '6949611']], ['Maniola_jurtina', ['6947151', '6947309']]]
Proceed?y
Aricia_agestis Exon_15 50_frame0 0.271 
SIVETNINLEQAKSSECVRSRSSRASSLGEADEDPPPDDKLDGHTDFECNNN
Danaus_plexippus Exon_15 1_frame2 1.691 
DISQSSIQKSRRDYAPAVGASKADMLPAADNHYQVHKEKHLANILNFTP
Maniola_jurtina Exon_15 6_frame0 1.695 
DGTLSVKIV
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ ---------------SIVETNINLEQAKSSKCVRSSSSRAGSS

Choose Species 1 - 31
Exon_11 Match
[['Aricia_agestis', ['26875252', '26875451']], ['Danaus_plexippus', ['26875252', '26875451']], ['Maniola_jurtina', ['26875252', '26875451']]]
Exon_12 Match
[['Aricia_agestis', ['26878945', '26879145']], ['Danaus_plexippus', ['26878945', '26879145']], ['Maniola_jurtina', ['26878945', '26879145']]]
Exon_13 Match
[['Aricia_agestis', ['26879822', '26879993']], ['Danaus_plexippus', ['26879822', '26879993']], ['Maniola_jurtina', ['26879822', '26879993']]]
Exon_14 Match
[['Aricia_agestis', ['26880495', '26880593']], ['Danaus_plexippus', ['26880495', '26880593']], ['Maniola_jurtina', ['26880495', '26880593']]]
Exon_15 mismatch
[['Aricia_agestis', ['26880844', '26881005']], ['Danaus_plexippus', ['26879844', '26879993']], ['Maniola_jurtina', ['26880844', '26881005']]]
Proceed?y
Aricia_agestis Exon_15 22_frame0 0.14 
SIVETNINLEQAKSSECVRSRSSRAGSSLGEADEDPPPDDKLDGHTDFECNNN
Danaus_plexippus Exon_15 1_frame2 1.691 
DISQSSIQKSRRDYAPAVGASKADMLPAADNHYQVHKEKHLANILNFTP
M

Exon_1 Match
[['Aricia_agestis', ['59981', '60077']], ['Danaus_plexippus', ['59981', '60077']], ['Maniola_jurtina', ['59981', '60077']]]
Exon_2 Match
[['Aricia_agestis', ['58718', '59268']], ['Danaus_plexippus', ['58718', '59268']], ['Maniola_jurtina', ['58718', '59268']]]
Exon_3 Match
[['Aricia_agestis', ['55197', '55350']], ['Danaus_plexippus', ['55197', '55350']], ['Maniola_jurtina', ['55197', '55350']]]
Exon_4 Match
[['Aricia_agestis', ['51594', '51835']], ['Danaus_plexippus', ['51594', '51835']], ['Maniola_jurtina', ['51594', '51835']]]
Exon_5 Match
[['Aricia_agestis', ['50598', '50896']], ['Danaus_plexippus', ['50598', '50896']], ['Maniola_jurtina', ['50598', '50896']]]
Exon_6 Match
[['Aricia_agestis', ['48762', '49122']], ['Danaus_plexippus', ['48762', '49122']], ['Maniola_jurtina', ['48762', '49122']]]
Exon_7 Match
[['Aricia_agestis', ['47209', '47400']], ['Danaus_plexippus', ['47209', '47400']], ['Maniola_jurtina', ['47209', '47400']]]
Exon_8 Match
[['Aricia_agestis', ['45206'

In [ ]:
1